Download from Wikipedia a table that lists the companies that comprise the S&P 500. From this table, extract the list of ticker symbols. Download the historical close prices from yahoo finance & drop all columnns with missing values.

From the data select the following 10 companies (these are the first 10 companies with no missing data):
"MMM","AOS","ABT","ADM","ADBE","ADP","AES","AFL","A","AKAM"

-So, our "new" dataset will consist of monthly returns you had computed in the last assignment, but just for these 10 companies.

We assume that the risk-free rate of return is zero.

In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
import requests
from io import StringIO

start_date = "2000-01-01"
end_date   = "2022-12-31"

## Import required prices data as done in Assignment 1
# 1. Download page with browser-like headers (to avoid 403)
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/120.0.0.0 Safari/537.36"
}
html = requests.get(url, headers=headers).text

# 2. Read all tables from the HTML
sp500_tables = pd.read_html(StringIO(html))

# 3. Finding the table that contains the 'Symbol' column
sp500_table = None
for i, tbl in enumerate(sp500_tables):
    print(f"Table {i} columns:", list(tbl.columns))
    if "Symbol" in tbl.columns:
        sp500_table = tbl
        break

# 4. Extract ticker list
sp500_tickers = sp500_table["Symbol"].unique().tolist()

# 5. Download daily prices (Close already adjusted because auto_adjust=True)
sp500_data = yf.download(
    sp500_tickers,
    start=start_date,
    end=end_date,
    auto_adjust=True
)

# 6. Take only the 'Close' level (adjusted close prices)
sp500_prices = sp500_data["Close"]

# 7.Drop columns that are entirely NaN
sp500_prices = sp500_prices.dropna(axis=1, how="all")

# 8.Drop companies with more than 100 missing observations
missing_counts = sp500_prices.isna().sum()
valid_cols = missing_counts[missing_counts <= 100].index
sp500_prices = sp500_prices[valid_cols]

# 9.Drop rows where all remaining tickers are NaN
sp500_prices = sp500_prices.dropna(how="all")
sp500_prices.head()

Table 0 columns: [0, 1]
Table 1 columns: ['Symbol', 'Security', 'GICS Sector', 'GICS Sub-Industry', 'Headquarters Location', 'Date added', 'CIK', 'Founded']


[*********************100%***********************]  503 of 503 completed

8 Failed downloads:
['BRK.B']: YFTzMissingError('possibly delisted; no timezone found')
['Q', 'SOLS', 'GEV', 'SOLV', 'VLTO', 'KVUE']: YFPricesMissingError('possibly delisted; no price data found  (1d 2000-01-01 -> 2022-12-31) (Yahoo error = "Data doesn\'t exist for startDate = 946702800, endDate = 1672462800")')
['BF.B']: YFPricesMissingError('possibly delisted; no price data found  (1d 2000-01-01 -> 2022-12-31)')


Ticker,A,AAPL,ABT,ACGL,ADBE,ADI,ADM,ADP,ADSK,AEE,...,WMB,WMT,WRB,WSM,WST,WY,XEL,XOM,YUM,ZBRA
Date,,,,,,,,,,,,,,,,,,,,,
2000-01-03,43.113316,0.839280,8.134053,1.215037,16.274673,27.737501,6.064440,23.966087,8.052906,10.188300,...,8.859447,14.239621,0.790358,6.933905,5.400741,11.024057,6.628227,17.255518,4.545775,25.027779
2000-01-04,39.819927,0.768521,7.901651,1.208433,14.909400,26.334282,6.001273,23.966087,7.660816,10.188300,...,8.662971,13.706800,0.766039,6.715355,5.456530,10.609920,6.780850,16.925013,4.454403,24.666668
2000-01-05,37.349907,0.779767,7.887124,1.320692,15.204177,26.718735,5.906516,23.735641,7.178246,10.582433,...,9.198819,13.427073,0.753880,6.755093,5.434212,11.171964,7.042491,17.847694,4.477246,25.138889
2000-01-06,35.927765,0.712287,8.163104,1.307485,15.328290,25.988285,5.938102,24.052509,6.740915,10.543022,...,9.395301,13.573596,0.797654,5.404074,5.378420,11.694578,6.977080,18.770376,4.439174,23.777779
2000-01-07,38.921738,0.746027,8.250254,1.380124,16.072983,26.718735,6.032856,24.599800,7.540174,10.700676,...,9.645367,14.599284,0.785495,5.314669,5.434212,11.310016,6.977080,18.715292,4.340186,23.513889


In [2]:
#Resample for monthly prices and 10 selected stocks
df_prices = sp500_prices[["MMM","AOS","ABT","ADM","ADBE","ADP","AES","AFL","A","AKAM"]].resample('M').last()
df_rets = df_prices.pct_change()[1:] #Monthly simple returns
df_rets

C:\Users\g\AppData\Local\Temp\ipykernel_14212\2113266644.py:2: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df_prices = sp500_prices[["MMM","AOS","ABT","ADM","ADBE","ADP","AES","AFL","A","AKAM"]].resample('M').last()


Ticker,MMM,AOS,ABT,ADM,ADBE,ADP,AES,AFL,A,AKAM
Date,,,,,,,,,,
2000-02-29,-0.051977,-0.126582,0.013436,-0.140034,0.852440,-0.081686,0.046021,-0.156653,0.569405,0.048670
2000-03-31,0.004252,0.043478,0.066288,0.024844,0.091567,0.109849,-0.060402,0.246153,0.001203,-0.384450
2000-04-30,-0.021877,0.149574,0.097843,-0.036363,0.086468,0.115285,0.142063,0.071331,-0.147837,-0.385154
2000-05-31,-0.003685,0.021277,0.058536,0.207111,-0.069251,0.022067,-0.029881,0.060746,-0.169253,-0.324905
2000-06-30,-0.032069,-0.003720,0.095238,-0.178011,0.155146,-0.024649,0.045845,-0.111246,0.001698,0.778792
...,...,...,...,...,...,...,...,...,...,...
2022-08-31,-0.122967,-0.107792,-0.056872,0.066812,-0.089437,0.013644,0.145365,0.043709,-0.043624,-0.061734
2022-09-30,-0.111379,-0.139416,-0.057379,-0.084651,-0.263068,-0.070548,-0.111984,-0.054190,-0.052242,-0.110323
2022-10-31,0.138371,0.134180,0.027355,0.205469,0.157340,0.068571,0.164475,0.158541,0.140185,0.099726


# Analysis without short selling constraints (short selling is allowed)
Choose the estimation window to be T<sup>est</sup> = 60 months of monthly returns. Call this the estimation sample. Use the estimation sample to compute the following two portfolio strategies:

a. mean-variance portfolio (MVP) without constraints on the size of the weight (assume that a risk-free rate is available, with the risk-free rate equal to zero);
    
b. global minimum variance (GMV) portfolio without constraints on the size of the weight.

For each of the two portfolios, rescale the weights in the risky assets so that they sum to 1; that is, you are fully invested" in just the risky assets.

In [3]:
estim = 60

def MVP_weights(returns, estim_window, first = True):
    """
    returns the mean variance portfolio weights
    first = True implies we calculate using the first estim_window observations
    first = False implies we calculate using the last estim_window observations
    """
    if first:
        sample = returns[:estim_window]
    else:
        sample = returns[-estim_window:]
    
    mu = sample.mean()
    sigma = sample.cov()

    numerator = np.linalg.inv(sigma) @ mu
    denominator = np.ones(len(mu)) @ numerator

    return (numerator / denominator).round(2)

def GMV_weights(returns, estim_window, first = True):
    """
    returns the global minimum variance portfolio weights
    first = True implies we calculate using the first estim_window observations
    first = False implies we calculate using the last estim_window observations
    """
    if first:
        sample = returns[:estim_window]
    else:
        sample = returns[-estim_window:]

    sigma = sample.cov()
    n = returns.shape[1]
    numerator = np.linalg.solve(sigma, np.ones(n))
    return (numerator / np.sum(numerator)).round(2)

wt_mvp = MVP_weights(df_rets, estim, first=True)  #Use first=Flase for last T_est observations
wt_gmv = GMV_weights(df_rets, estim, first=True)  #Use first=Flase for last T_est observations


#Create results dataframe
df_results = pd.DataFrame(
    {
        'MVP Weights': wt_mvp,
        'GMV Weights': wt_gmv
    },
    index=df_rets.columns
)
#add total row to show that weights sum to 1
df_results.loc['Total Weights'] = df_results.sum(axis=0).round(2)
df_results

,MVP Weights,GMV Weights
Ticker,,
MMM,0.31,0.34
AOS,0.15,0.10
ABT,0.24,0.30
ADM,0.40,0.06
ADBE,0.27,0.03
ADP,-0.25,0.09
AES,0.04,0.02
AFL,0.02,0.08
A,-0.18,0.00


Now use a rolling window of T<sup>est</sup> = 60 months to estimate the portfolio weights for the two strategies listed above for each of the T - T<sup>est</sup> months.

In [4]:
estim_2 = 60
# A) Rolling Weights of Mean Variance Portfolio

def MVP_rolling_weights(returns, rolling_window):
    """
    returns the mean variance portfolio weights for a rolling window
    """
    n_assets = returns.shape[1]
    n_periods = returns.shape[0] 

    #array to store weights in for loop
    mvp_weights = np.full((n_periods, n_assets), np.nan)
    
    for i in range(rolling_window, n_periods):

        #resample to rolling window
        sample = returns.iloc[i - rolling_window:i]
        mu = sample.mean()
        sigma = sample.cov()
        
        numerator = np.linalg.inv(sigma) @ mu
        denominator = np.ones(len(mu)) @ numerator
        
        mvp_weights[i] = (numerator / denominator)

    return pd.DataFrame(mvp_weights, columns=returns.columns, index=returns.index)

wt_mvp = MVP_rolling_weights(df_rets, estim_2).dropna()
wt_mvp['Total weights'] = wt_mvp.sum(axis=1).round(2)
wt_mvp


Ticker,MMM,AOS,ABT,ADM,ADBE,ADP,AES,AFL,A,AKAM,Total weights
Date,,,,,,,,,,,
2005-02-28,0.307004,0.153489,0.236078,0.403321,0.270471,-0.250642,0.038042,0.024036,-0.179817,-0.001983,1.0
2005-03-31,0.326571,0.158997,0.227712,0.351975,0.162405,-0.162818,0.043949,0.069106,-0.181398,0.003500,1.0
2005-04-30,0.405559,0.184817,0.217775,0.412999,0.205943,-0.195133,0.053826,-0.067531,-0.234815,0.016559,1.0
2005-05-31,0.370288,0.169687,0.302905,0.259169,0.119814,-0.217458,0.048162,0.112800,-0.189431,0.024065,1.0
2005-06-30,0.370612,0.182884,0.276221,0.269249,0.138794,-0.247065,0.039235,0.118080,-0.179272,0.031262,1.0
...,...,...,...,...,...,...,...,...,...,...,...
2022-08-31,-1.303842,-0.267136,0.772434,0.486913,0.329618,0.516556,-0.154242,-0.003842,0.584387,0.039154,1.0
2022-09-30,-1.283041,-0.286634,0.746668,0.518453,0.290888,0.643356,-0.157878,-0.031230,0.515952,0.043463,1.0
2022-10-31,-1.527634,-0.401820,0.815950,0.524974,0.248261,0.748885,-0.204840,0.012253,0.762062,0.021909,1.0


In [5]:
# B) Rolling Weights of Global Minimum Variance Portfolio

def GMV_rolling_weights(returns, rolling_window):
    """
    returns the global minimum variance portfolio weights for a rolling window
    """
    n_assets = returns.shape[1]
    n_periods = returns.shape[0] 

    #array to store weights in for loop
    gmv_weights = np.full((n_periods, n_assets), np.nan)
    
    for i in range(rolling_window, n_periods):

        #resample to rolling window
        sample = returns.iloc[i - rolling_window:i]
        sigma = sample.cov()
        
        numerator = np.linalg.solve(sigma, np.ones(n_assets))
        gmv_weights[i] = (numerator / np.sum(numerator))

    return pd.DataFrame(gmv_weights, columns=returns.columns, index=returns.index)

wt_gmv = GMV_rolling_weights(df_rets, estim_2).dropna()
wt_gmv['Total weights'] = wt_gmv.sum(axis=1).round(2)
wt_gmv


Ticker,MMM,AOS,ABT,ADM,ADBE,ADP,AES,AFL,A,AKAM,Total weights
Date,,,,,,,,,,,
2005-02-28,0.336896,0.104300,0.304425,0.055130,0.030109,0.085953,0.021370,0.079242,0.004852,-0.022278,1.0
2005-03-31,0.339982,0.114335,0.287954,0.043529,-0.011086,0.125356,0.018376,0.105576,-0.009318,-0.014703,1.0
2005-04-30,0.300812,0.111240,0.283741,0.027026,-0.019051,0.141704,0.012420,0.160141,0.000226,-0.018260,1.0
2005-05-31,0.270909,0.128246,0.291900,-0.005095,-0.018808,0.183106,0.012640,0.170285,-0.010016,-0.023167,1.0
2005-06-30,0.254060,0.128797,0.291445,0.012739,-0.016566,0.181707,0.011912,0.173709,-0.013872,-0.023932,1.0
...,...,...,...,...,...,...,...,...,...,...,...
2022-08-31,0.003398,-0.182658,0.358340,0.242732,0.039779,0.248541,-0.035553,0.068606,0.059210,0.197605,1.0
2022-09-30,-0.011828,-0.188186,0.356347,0.253427,0.032440,0.279289,-0.033467,0.064965,0.050814,0.196199,1.0
2022-10-31,-0.014524,-0.204820,0.372406,0.240901,-0.025612,0.303641,-0.042814,0.073509,0.094181,0.203132,1.0


Use the time-series of portfolios weights for each of the two portfolio strategies, to compute the out-of-sample portfolio returns. That is, for each of the two portfolio strategies that you estimate at each date t, compute its out-of-sample return in month t + 1.

In [6]:
def out_of_sample_portfolio_returns(returns, weights_function, **kwargs):
    """
    computes out-of-sample portfolio returns given a returns DataFrame and a weights function
    weights_function should be a function that takes returns and **kwargs as input and returns weights DataFrame
    """
    weights = weights_function(returns, **kwargs).dropna()
    sample_rets = returns[weights.index[0]:]

    #calculate out of sample portfolio returns as: weights(t-1) * returns(t)
    portfolio_rets = (weights.shift() * sample_rets).sum(axis=1)  
    
    #skip first row as they do not have any prior weights, so cannot be used for out-of-sample returns
    return portfolio_rets.iloc[1:] 

portfolio_rets_mvp = out_of_sample_portfolio_returns(df_rets, MVP_rolling_weights, rolling_window=60).rename("MVP Returns")
portfolio_rets_gmv = out_of_sample_portfolio_returns(df_rets, GMV_rolling_weights, rolling_window=60).rename("GMV Returns")

df_portfolio_rets = pd.concat([portfolio_rets_mvp, portfolio_rets_gmv], axis=1)
df_portfolio_rets

,MVP Returns,GMV Returns
Date,,
2005-03-31,0.055726,0.023009
2005-04-30,-0.112930,-0.023636
2005-05-31,0.044743,0.009805
2005-06-30,-0.013568,-0.024319
2005-07-31,-0.000573,0.011154
...,...,...
2022-08-31,0.116340,-0.006206
2022-09-30,-0.043597,-0.068651
2022-10-31,0.046583,0.090671


Now, compute the Sharpe ratio of the out-of-sample returns for the two portfolio strategies. Which strategy has the higher Sharpe ratio? Why?

In [7]:
def sharpe_ratio(returns, risk_free_rate=0.0, periods_per_year=12):
    """
    computes the annualized Sharpe ratio of a returns series
    risk free rate is in decimal form (e.g., 0.03 for 3%)
    perods_per_year: number of return periods in a year (e.g., 12 for monthly returns, quarterly=4, daily=252)
    """

    if isinstance (returns, pd.DataFrame):
        return returns.aggregate(sharpe_ratio, risk_free_rate=risk_free_rate, periods_per_year=periods_per_year)

    
    elif isinstance (returns, pd.Series):
        ann_return = returns.mean() * periods_per_year
        ann_volatility = returns.std() * np.sqrt(periods_per_year)
        return (ann_return - risk_free_rate) / ann_volatility

sharpe_ratio(df_portfolio_rets).round(4).rename("Sharpe Ratio")
    

MVP Returns    0.6066
GMV Returns    0.8108
Name: Sharpe Ratio, dtype: float64

The GMV portfolio exhibits better out-of-sample Sharpe ratios due to lower estimation error.
Weight Formulation (with rf = 0):
$$W_{MVP} = \frac{\Sigma^{-1}\mu}{1'\Sigma^{-1}\mu}$$

$$W_{GMV} = \frac{\Sigma^{-1}1}{1'\Sigma^{-1}1}$$

Expected returns μ have much larger estimation error than the covariance matrix Σ. For typical estimation windows (T=60 months), this creates highly unstable MVP weights. Since GMV avoids estimating returns entirely, it produces more stable weights and better out-of-sample performance.


# Analysis with short selling constraint (Long only portfolio)
Using the estimation window of T<sup>est</sup> = 60 months of monthly returns, now compute the following two portfolio strategies:

a. mean-variance portfolio with nonnegativity constraints on the weights (when a risk-free rate is available, and set this rate to 0); we will refer to this portfolio as "MVP-C."

b. global minimum variance (GMV) portfolio with nonnegativity constraints; we will refer to this portfolio as "GMV-C". 

For each of the two portfolios, rescale the weights in the risky assets so that they sum to 1; that is, you are fully invested" in just the risky assets.

In [8]:
# we will use a quadratic optimizer to solve for the weights
from scipy.optimize import minimize

#We'll modify the funciton from Q 1.2 for an additional argument: nonnegativity constraints
def MVP_weights(returns, estim_window, rf = 0.0, first = True, long_only = False, GMV = False):
    """
    returns the mean variance portfolio weights when GMV = False (default)
    returns the global minimum variance portfolio weights when GMV = True
    first = True: implies we calculate using the first estim_window observations
    first = False: implies we calculate using the last estim_window observations
    long_only = True: implies nonnegativity constraints on weights
    """
    if first:
        sample = returns[:estim_window]
    else:
        sample = returns[-estim_window:]
    
    if GMV: #GMV does not use expected returns to calculate weights
        mu = np.ones(sample.shape[1])
    else:
        mu = sample.mean().values

    sigma = sample.cov().values

    #In case of no short selling:
    if long_only:
        def negative_sharpe_ratio(weights):
            portfolio_return = weights @ mu
            porfolio_volatility = np.sqrt(weights @sigma @ weights)
            return - (portfolio_return - rf) / porfolio_volatility
        
        n = sample.shape[1] #number of assets
        initial_guess = np.repeat(1/n, n)
        bounds = ((0,1),) * n  #short selling not allowed (weight constraint for the optimizer)
        #define constraint that weights sum to 1
        weights_sum_to_1 = {
            'type': 'eq',
            'fun': lambda weights: np.sum(weights) - 1
        }

        #quadratic optimization to minimize negative Sharpe ratio (bascially maximizing sharpe ratio)
        results = minimize(negative_sharpe_ratio, initial_guess, method = 'SLSQP', 
                           bounds=bounds, constraints=weights_sum_to_1)
        
        return results.x
    
    #If no constraints on weights are implied
    else: 
        numerator = np.linalg.inv(sigma) @ (mu - rf)
        denominator = np.ones(len(mu)) @ numerator
        return (numerator / denominator).round(2)
    
mvp_c= MVP_weights(df_rets, estim_window=60, long_only=True)
gmv_c= MVP_weights(df_rets, estim_window=60, long_only=True, GMV=True)

#Create results dataframe
df_results = pd.DataFrame({
    'MVP-C Weights': mvp_c,
    'GMV-C Weights': gmv_c
},
    index=df_rets.columns
)
df_results.loc['Total Weight'] = df_results.sum(axis=0)
df_results.round(4)

,MVP-C Weights,GMV-C Weights
Ticker,,
MMM,0.2329,0.3262
AOS,0.0993,0.1020
ABT,0.2081,0.2933
ADM,0.2427,0.0589
ADBE,0.1142,0.0262
ADP,0.0000,0.0829
AES,0.0017,0.0097
AFL,0.1011,0.1008
A,0.0000,0.0000


Now use a rolling window of T<sup>est</sup> = 60 months to estimate the portfolio weights for the two strategies listed above for each of the T - T<sup>est</sup> months.

In [9]:
# we will modify the funciton in Q 1.3 for nonnegativity constraints

def MVP_rolling_weights(returns, rolling_window, rf=0.0,long_only = False, GMV = False):
    """
    returns the mean variance portfolio weights for a rolling window if GMV = False (default)
    returns the global minimum variance portfolio weights for a rolling window if GMV = True
    long_only = True: implies nonnegativity constraints on weights
    """
    n_assets = returns.shape[1]
    n_periods = returns.shape[0] 

    #array to store weights in for loop
    weights = np.full((n_periods, n_assets), np.nan)

    for i in range(rolling_window, n_periods):
        #resample to rolling window
        sample = returns.iloc[i - rolling_window:i]  
        weights[i] = MVP_weights(returns=sample, estim_window=rolling_window,
                                     rf=rf, long_only=long_only, GMV=GMV) 

    return pd.DataFrame(weights, columns=returns.columns, index=returns.index)

mvp_c_rolling = MVP_rolling_weights(df_rets, rolling_window=60, long_only=True).dropna()
mvp_c_rolling['Total weight'] = mvp_c_rolling.sum(axis=1).round(2)
mvp_c_rolling

Ticker,MMM,AOS,ABT,ADM,ADBE,ADP,AES,AFL,A,AKAM,Total weight
Date,,,,,,,,,,,
2005-02-28,2.329202e-01,9.931860e-02,0.208077,0.242676,1.141615e-01,0.000000e+00,0.001714,1.011325e-01,0.000000e+00,0.000000e+00,1.0
2005-03-31,2.709668e-01,1.040720e-01,0.223743,0.216963,6.969928e-03,0.000000e+00,0.018163,1.591229e-01,0.000000e+00,0.000000e+00,1.0
2005-04-30,3.161932e-01,1.176922e-01,0.220639,0.233885,1.858410e-02,2.703397e-15,0.022417,7.058963e-02,4.820266e-15,2.140978e-15,1.0
2005-05-31,2.894769e-01,1.155910e-01,0.267156,0.138880,0.000000e+00,1.914333e-15,0.019552,1.693446e-01,2.338194e-15,1.596673e-15,1.0
2005-06-30,3.096989e-01,1.341580e-01,0.246658,0.126575,1.206655e-03,0.000000e+00,0.017251,1.644523e-01,0.000000e+00,0.000000e+00,1.0
...,...,...,...,...,...,...,...,...,...,...,...
2022-08-31,0.000000e+00,5.358644e-16,0.338371,0.234177,1.443962e-01,1.517280e-01,0.023648,2.472529e-16,2.100593e-17,1.076795e-01,1.0
2022-09-30,3.498449e-16,2.021766e-16,0.250634,0.250166,5.454597e-02,2.516403e-01,0.078561,1.746650e-16,1.278816e-16,1.144521e-01,1.0
2022-10-31,0.000000e+00,3.491131e-17,0.291375,0.252514,4.038653e-17,3.014386e-01,0.081038,0.000000e+00,0.000000e+00,7.363495e-02,1.0


In [10]:
# use the same function with GMV = True for rolling GMV weights 
gmv_c_rolling = MVP_rolling_weights(df_rets, rolling_window=60, long_only=True, GMV=True).dropna()
gmv_c_rolling['Total weight'] = gmv_c_rolling.sum(axis=1).round(2)
gmv_c_rolling

Ticker,MMM,AOS,ABT,ADM,ADBE,ADP,AES,AFL,A,AKAM,Total weight
Date,,,,,,,,,,,
2005-02-28,3.261593e-01,1.020497e-01,0.293273,0.058945,2.621470e-02,0.082929,9.677786e-03,0.100751,2.877864e-12,0.000000e+00,1.0
2005-03-31,3.221340e-01,1.076565e-01,0.280260,0.055666,0.000000e+00,0.102416,8.640865e-03,0.123226,0.000000e+00,0.000000e+00,1.0
2005-04-30,2.833606e-01,1.069321e-01,0.271491,0.045107,1.896391e-13,0.119074,1.487161e-03,0.172549,2.044462e-14,1.526138e-12,1.0
2005-05-31,2.530445e-01,1.162813e-01,0.280378,0.016973,0.000000e+00,0.141879,1.167415e-13,0.191445,1.738910e-14,0.000000e+00,1.0
2005-06-30,2.422606e-01,1.140829e-01,0.282264,0.028827,0.000000e+00,0.137602,6.800696e-13,0.194963,0.000000e+00,0.000000e+00,1.0
...,...,...,...,...,...,...,...,...,...,...,...
2022-08-31,1.390882e-15,2.891892e-14,0.314228,0.200328,4.780939e-03,0.197232,7.142320e-03,0.065807,0.000000e+00,2.104812e-01,1.0
2022-09-30,0.000000e+00,5.694230e-15,0.297072,0.206998,0.000000e+00,0.213009,1.466691e-02,0.060578,0.000000e+00,2.076769e-01,1.0
2022-10-31,1.023390e-14,9.026999e-14,0.316826,0.204726,1.404044e-14,0.218946,0.000000e+00,0.071480,0.000000e+00,1.880217e-01,1.0


Use the time-series of portfolios weights for each of the two portfolio strategies, to compute the out-of-sample portfolio returns. That is, for each of the two portfolio strategies that you estimate at each date t, compute its out-of-sample return in month t + 1.

In [11]:
# We'll use the funciton from Q 1.4 with the new rolling weights functions defined in Q 2.2

portfolio_rets_mvp_c = out_of_sample_portfolio_returns(df_rets, MVP_rolling_weights, rolling_window=60, long_only=True).rename("MVP-C Returns")
portfolio_rets_gmv_c = out_of_sample_portfolio_returns(df_rets, MVP_rolling_weights, rolling_window=60, long_only=True, GMV=True).rename("GMV-C Returns")
df_portfolio_rets_c = pd.concat([portfolio_rets_mvp_c, portfolio_rets_gmv_c], axis=1)
df_portfolio_rets_c

,MVP-C Returns,GMV-C Returns
Date,,
2005-03-31,0.029513,0.025488
2005-04-30,-0.061178,-0.025775
2005-05-31,0.037689,0.017933
2005-06-30,-0.009592,-0.023292
2005-07-31,0.018987,0.017044
...,...,...
2022-08-31,-0.018905,-0.011309
2022-09-30,-0.102457,-0.077747
2022-10-31,0.108430,0.097991


Now, compute the Sharpe ratio of the out-of-sample returns for the two portfolio strategies. "MVP-C" and "GMV-C." Which strategy has the higher Sharpe ratio? Why?

In [12]:
# We will use the same Sharpe ratio function defined in Q 1.5
sharpe_ratio(df_portfolio_rets_c).round(4).rename("Sharpe Ratio")

MVP-C Returns    0.8413
GMV-C Returns    0.8530
Name: Sharpe Ratio, dtype: float64

The GMV-C strategy still outperforms MVP-C due to the fact that expected returns μ have much larger estimation error than the covariance matrix Σ.
However, by introducing the "no short selling" constraint, the performance of MVP-C has improved a lot compared to plain MVP strategy

Finally, out of the four strategies you have considered in the two questions, "MVP", "GMV, ", "MVP-C" and "GMV-C," which strategy has the highest Sharpe ratio. Why do you think this strategy performs the best?

In [13]:
portfolio_rets_mvp = out_of_sample_portfolio_returns(df_rets, MVP_rolling_weights, rolling_window=60, long_only=False).rename("MVP Returns")
portfolio_rets_gmv = out_of_sample_portfolio_returns(df_rets, MVP_rolling_weights, rolling_window=60, long_only=False, GMV=True).rename("GMV Returns")

df_portfolio_rets_final = pd.concat([portfolio_rets_mvp, portfolio_rets_gmv, df_portfolio_rets_c], axis=1)
sharpe_ratio(df_portfolio_rets_final).round(4).rename("Sharpe Ratio")

MVP Returns      0.6082
GMV Returns      0.8151
MVP-C Returns    0.8413
GMV-C Returns    0.8530
Name: Sharpe Ratio, dtype: float64

GMV-C is the best performing strategy out of the 4. The resoning remains same as explained above. The adition of "no short selling" constraint did not improve the performance of GMV by a large factor. However, the MVP portfolio had a large performance imporvement when subject to "no short selling".